### Analysis and detection all inages from test set


Overview. In this notebook I simply create a dataframe with total amount of all classes such as person, car, chair, 
horse, boat etc with defined probability. My goal is to plot the distribution and count all classes in test set with given probability. The way how do I plot in you mau find here <https://github.com/veronikayurchuk/ComputerVision/blob/master/classes_distr_by_probs.nojekyll.md>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 20)
plt.rcParams.update({'font.size': 22})

import os
import operator
from collections import Counter

caffe_root = '/home/veronika/materials/cv/detection/caffe/examples/'  

import sys
#sys.path.append("/home/veronika/materials/cv/detection/caffe/build/tools/caffe")
import sys
sys.path.append('/home/veronika/materials/cv/detection/caffe/python')
sys.path.remove('/home/veronika/caffe/python')

import caffe

from google.protobuf import text_format
from caffe.proto import caffe_pb2

# load PASCAL VOC labels
labelmap_file = '/home/veronika/materials/cv/detection/git/labelmap_voc.prototxt'
file = open(labelmap_file, 'r')
labelmap = caffe_pb2.LabelMap()
text_format.Merge(str(file.read()), labelmap)

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [4]:
def get_labelname(labelmap, labels):
    num_labels = len(labelmap.item)
    labelnames = []
    if type(labels) is not list:
        labels = [labels]
    for label in labels:
        found = False
        for i in xrange(0, num_labels):
            if label == labelmap.item[i].label:
                found = True
                labelnames.append(labelmap.item[i].display_name)
                break
        assert found == True
    return labelnames

In [5]:
caffe.set_device(0)
caffe.set_mode_gpu()
model_def = '/home/veronika/materials/cv/detection/models_trained/VGGNet/deploy.prototxt'
model_weights = '/home/veronika/materials/cv/detection/models_trained/VGGNet/VGG_VOC0712_SSD_300x300_iter_60000.caffemodel'

net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2, 0, 1))
transformer.set_mean('data', np.array([104,117,123])) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB

In [6]:
labels = pd.read_csv("/home/veronika/materials/cv/cv_organizer/mydata/test_labels.csv",
                    sep = " ", header = None)

In [7]:
def get_object_descr(path_to_test, conf_level):
    descr_total = {}
    for i in range(labels[0].shape[0]):
        #path_to_test = path_to_img
        image = caffe.io.load_image(os.path.join(path_to_test, labels[0][i]))
        transformed_image = transformer.preprocess('data', image)
        net.blobs['data'].data[...] = transformed_image

        # Forward pass.
        detections = net.forward()['detection_out']
    
        det_label = detections[0,0,:,1]
        det_conf = detections[0,0,:,2]

        # Get detections with confidence higher than 0.6.
        top_indices = [i for i, conf in enumerate(det_conf) if conf >= conf_level]

        top_conf = det_conf[top_indices]
        top_label_indices = det_label[top_indices].tolist()
        top_labels = get_labelname(labelmap, top_label_indices)
        #descr = Counter(dict((i,top_labels.count(i)) for i in set(top_labels)))
        descr = Counter(top_labels)
        descr_total = descr + Counter(descr_total)
        print("Done!")
    return(descr_total)

In [8]:
path_to_test = "/home/veronika/materials/cv/cv_organizer/mydata/test/"
object_desc_02 = get_object_descr(path_to_test, 0.2)
object_desc_04 = get_object_descr(path_to_test, 0.4)
object_desc_06 = get_object_descr(path_to_test, 0.6)
object_desc_08 = get_object_descr(path_to_test, 0.8)

In [46]:
object_desc_08

Counter({u'aeroplane': 8,
         u'bicycle': 54,
         u'bird': 6,
         u'boat': 31,
         u'bottle': 3,
         u'bus': 5,
         u'car': 62,
         u'cat': 1,
         u'chair': 71,
         u'diningtable': 19,
         u'dog': 16,
         u'horse': 49,
         u'motorbike': 9,
         u'person': 2158,
         u'pottedplant': 30,
         u'sheep': 1,
         u'sofa': 14,
         u'train': 6,
         u'tvmonitor': 18})

In [47]:
object_desc_02_df = pd.DataFrame(object_desc_02.items(), columns=['ClassType', 'Prob02'])
object_desc_04_df = pd.DataFrame(object_desc_04.items(), columns=['ClassType', 'Prob04'])
object_desc_06_df = pd.DataFrame(object_desc_06.items(), columns=['ClassType', 'Prob06'])
#object_desc_08_df = pd.DataFrame(object_desc_08.items(), columns=['ClassType', 'Prob08'])

In [65]:
totaldata = pd.merge(object_desc_02_df, object_desc_04_df, how='outer')
totaldata = pd.merge(totaldata, object_desc_06_df, how = "outer")
#totaldata = pd.merge(totaldata, object_desc_08_df, how = "outer")
totaldata

,ClassType,Prob02,Prob04,Prob06
0,sheep,17,7,4
1,bottle,42,9,3
2,horse,101,76,64
3,bicycle,168,92,61
4,motorbike,50,25,14
5,cow,17,5,1
6,bus,21,13,11
7,dog,74,46,29
8,cat,4,1,1
9,person,4688,3367,2699


In [66]:
totaldata_melted = pd.melt(totaldata, id_vars=['ClassType'], value_vars=['Prob02', 'Prob04', 'Prob06'],
                          var_name='Probs', value_name='Amount')
totaldata_melted


,ClassType,Probs,Amount
0,sheep,Prob02,17
1,bottle,Prob02,42
2,horse,Prob02,101
3,bicycle,Prob02,168
4,motorbike,Prob02,50
5,cow,Prob02,17
6,bus,Prob02,21
7,dog,Prob02,74
8,cat,Prob02,4
9,person,Prob02,4688


In [43]:
totaldata_melted.to_csv("/home/veronika/materials/cv/detection/total_classes.csv")

In [ ]:
All details are here: https://github.com/weiliu89/caffe/tree/ssd/examples